In [1]:
import pandas as pd
import pickle
import os
from datetime import date

%matplotlib inline

## track_sleep

This function allows users to keep a simple weekly sleep log, with the option to print their average hours of sleep per week and save the log to update in subsequent weeks.


**Parameters**:

> *weekly_sleep* : list or dict
>> The current week's sleep log. Can be a list (of length 7) of hours of sleep for each night, a dict of Day : hours pairs for the week.
    
> *existing* : Pandas DataFrame (default None)
>> The existing sleep log to which the current week's data should be added, if passed.  Must be a DataFrame returned by `track_sleep`, or analogously formatted (rows = weeks, columns = days of the week).  If not passed, `track_sleep` returns a new sleep log

> *print_stats* : bool (default False)
>> If True, print the the average number of hours of sleep for the current week. If an existing sleep log is passed, print the average number of hours for each prior week. 

> *save_dir* : str (default None)
>> If passed, save the sleep log to the specified directory (named `sleep_log_<today's date>.p`) as a serialized `pickle` file.

**Returns**

> *sleep_log* : Pandas DataFrame
>> a DataFrame (*rows = weeks, columns = days of the week*) of the users's sleep log.

In [2]:
def track_sleep(weekly_sleep, existing=None, print_stats=False, save_dir=None):
    
    # explicitly order days of week
    days_ordered = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
    
    if existing is not None:
        # some checks to ensure existing sleep log is formatted correctly
        if not (isinstance(existing, pd.DataFrame) and all(existing.columns == days_ordered)
                and all([i.startswith('Week') and i[-1].isdigit() for i in existing.index])):
            raise TypeError("You may only add to sleep logs output by this program")
        
        # continue week numbering from prior week...
        week_num = int(existing.index.tolist()[-1].split()[-1]) + 1
    
    else:
        # ... or start from scratch
        week_num = 1
    
    # convert input options to dataframe, with some checks
    if isinstance(weekly_sleep, list):
        if len(weekly_sleep) != 7:
            raise ValueError("Your list must have one entry for each day of the week")
        weekly_sleep = pd.DataFrame([weekly_sleep], columns=days_ordered, 
                                    index=['Week '+ str(week_num)])
        
    elif isinstance(weekly_sleep, dict):
        weekly_sleep = pd.DataFrame(weekly_sleep, columns=days_ordered, 
                                    index=['Week '+ str(week_num)])
        
    else:
        raise TypeError("You must pass either a dictionary of (day : hours) pairs or a list of hours slept \
        (of length 7)")
    
    
    # if a previous sleep log is passed, append the current week's data
    if existing is not None:
        new_log = existing.append(weekly_sleep)
    
    # otherwise, create a new log
    else:
        new_log = weekly_sleep
    
    
    # print user's average hours of sleep for each week in their log
    if print_stats:
        message = 'You slept an average of {} hours this week'.format(round(float(new_log.tail(1).mean(axis=1)),2))
        
        if existing is not None:
            # print prior weeks' averages
            prev_wk = 3
            message += ', compared to {} hours last week'.format(round(float(new_log.iloc[-prev_wk, :].mean()),2))
            while prev_wk <= len(new_log):
                # format sentence properly
                if prev_wk == len(new_log):
                    message += ', and '
                else:
                    message += ', '
                message += '{0} hours {1} weeks ago'.format(round(float(new_log.iloc[-prev_wk, :].mean()),2), 
                                                            prev_wk-1)
                prev_wk += 1
        
        message += '.'
        print(message)
    
    if save_dir:
        # pickle the sleep log and save it
        with open(os.path.join(save_dir,'sleep_log_'+date.today().strftime("%m.%d.%y")+'.p'), 'wb') as f:
            pickle.dump(new_log, f)
            
    return new_log

In [3]:
w1_data = {'Monday' : 6, 'Tuesday' : 5.5, 'Wednesday' : 7, 'Thursday' : 6.5, 'Friday' : 8, 'Saturday' : 9, 
            'Sunday' : 9.25}
w1_log = track_sleep(w1_data)

w2_data = [6.25,7.75,4,5,6,3.73,9]
w1w2_log = track_sleep(w2_data, existing=w1_log)

w3_data = [9,8,5.75,10,11.25,9, 9.25]
w1w2w3_log = track_sleep(w3_data, existing=w1w2_log)

w4_data = {'Monday' : 7, 'Tuesday' : 5, 'Wednesday' : 7.75, 'Thursday' : 9.5, 'Friday' : 9, 'Saturday' : 1, 
            'Sunday' : 4.5}
w1w2w3w4_log = track_sleep(w4_data, existing=w1w2w3_log, print_stats=True)
w1w2w3w4_log

You slept an average of 6.25 hours this week, compared to 5.96 hours last week, 5.96 hours 2 weeks ago, and 7.32 hours 3 weeks ago.


,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
Week 1,6.00,5.50,7.00,6.5,8.00,9.00,9.25
Week 2,6.25,7.75,4.00,5.0,6.00,3.73,9.00
Week 3,9.00,8.00,5.75,10.0,11.25,9.00,9.25
Week 4,7.00,5.00,7.75,9.5,9.00,1.00,4.50
